# Descomposición por valores singulares y sistemas de recomendación.

Al igual que el análisis de componentes principales (PCA por sus siglas en inglés), la descomposición por valores singulares (SVD en inglés) es uno de los métodos más usuales para reducir la dimensión de nuestro conjunto de datos.

A diferencia del PCA, este método no requiere del uso de la matriz de **varianzas y covarianzas**.

+ **A favor:** Reduce la dimensión de nuestros datos, remueve el ruido que podrían contener.
+ **En contra:** Se pierde interpretación en los datos transformados.
+ **Tipo de datos:** Numéricos.

En estas notas, se expondrá la relación entre SVD y los sistemas de recomendación.
***

## Descomposición de la matriz de datos.
Sea $\mathbf{D_{m \times n}}$ una matriz de $m \times n$ la cual representa nuestro conjunto de datos. El método de descomposición por valores singulares nos dice que:

$$ \mathbf{D_{m \times n}} = \mathbf{U_{m \times m}} \mathbf{\Sigma_{m \times n}} \mathbf{V_{n \times n}^{T}}$$

en donde la matriz $\mathbf{\Sigma_{m \times n}}$ es una matriz con elementos igual a $0$, excepto posiblemente, en su diagonal. Estos valores distintos de cero, reciben el nombre de **valores singulares** y son igual a la raiz cuadrada de los eigenvalores de la matriz $\mathbf{D_{m \times n}} \mathbf{D_{m \times n}^{T}}$. Además, los valores en la diagonal están ordenados de mayor a menor.

## SVD en Python con numpy.
Es posible obtener factorización anterior utilizando el paquete **numpy**, en particular, la función **svd** del módulo **linalg**.

In [1]:
import numpy as np
datos = np.matrix([[1,2,3], [4,5,6]])
num_renglones = datos.shape[0] # m
num_columnas = datos.shape[1] #n
U, sig, VT = np.linalg.svd(datos)

print('m es igual a ' + str(num_renglones))
print('n es igual a ' + str(num_columnas))
print('La dimensión de U es ' + str(U.shape))
print('La dimensión de sigma es ' + str(sig.shape))
print('La dimensión de V^T es ' + str(VT.shape))
print(sig) #No es una matriz!

m es igual a 2
n es igual a 3
La dimensión de U es (2, 2)
La dimensión de sigma es (2,)
La dimensión de V^T es (3, 3)
[9.508032   0.77286964]


Como podemos observar, la función **svd**, en lugar de regresar la matriz $\mathbf{\Sigma_{m \times n}}$, nos regresa un vector que representa la diagonal de esta matriz. Esto se hace por cuestiones de ahorro en la memoria, recuerde que el resto de los elementos es igual a cero.

Para poder obtener $\mathbf{\Sigma_{m \times n}}$, utilizaremos las funciones **zeros** y **fill_diagonal** de **numpy**

In [5]:
def crea_matriz_sigma(sig, num_renglones, num_columnas):
    '''
    ENTRADA:
    sig: Arreglo con los valores singulares
    
    num_renglones: Entero que representa el número de renglones
    
    num_columnas: Entero que representa el número de columnas
    
    SALIDA:
    matriz con la diagonal formada por sig
    '''
    mat_sigma = np.zeros((num_renglones, num_columnas)) #matriz de ceros
    np.fill_diagonal(mat_sigma, sig) #Se modifica la diagonal
    
    return mat_sigma

In [6]:
mat_sigma = crea_matriz_sigma(sig, num_renglones, num_columnas)
print(mat_sigma)
print(mat_sigma.shape)

[[9.508032   0.         0.        ]
 [0.         0.77286964 0.        ]]
(2, 3)


Verifiquemos que la descomposición fue correcta

In [7]:
print('La matriz original es: ')
print(datos)
print('El producto de la descomposición es:')
print(U * mat_sigma * VT)

La matriz original es: 
[[1 2 3]
 [4 5 6]]
El producto de la descomposición es:
[[1. 2. 3.]
 [4. 5. 6.]]


## Reducción de la dimensión.
Algo que se ha observado en la práctica, es el hecho de que sólo un número, $k$, de los valores en la diagonal de la matriz $\mathbf{\Sigma_{m \times n}}$ son distintos de cero, esto indica que sólo un conjunto de $k$ atributos son considerados como importantes, los demás son ruido o repeticiones.

Así pues, podemos aproximar nuestro conjunto de datos utilizando sólomente una porción de la matriz $\mathbf{\Sigma_{m \times n}}$.

Una forma de elegir el número de valores singulares a utilizar, es estableciendo un umbral (e.g. $90\%$) para la "energía" explicada por estos valores. Esto se puede obtener sumando los cuadrados de cada valor en la diagonal ("energía" total) y considerar (agregando de mayor a menor valor) los $k$ valores singulares cuya "energía" agregada represente al menos cierto porcentaje de la energía total.

In [ ]:
def selecciona_k_importantes(sig, umbral = 0.90):
    '''
    ENTRADA:
    sig: Arreglo con los valores singulares (deben de estar ordenados de mayor a menor)
    
    umbral: Número en (0,1) que representa la cantidad mínima de "energía"
    explicada por los valores singulares
    
    SALIDA:
    Un arreglo con los k valores singulares más importantes
    '''
    
    #Calcula la energía total
    total = np.sum(sig**2.0)
    
    #Calcula las proporciones acumuladas (ordenadas de mayor a menor)
    proporciones = np.cumsum(sig**2.0) / total
    
    #para almacenar los k mejores
    k_mejores = []
    
    for k in range(0, len(proporciones)):
        if proporciones[k] < umbral:
            k_mejores.append(sig[k])
        #Ya que las proporciones acumuladas se ordenan de mayor a menor
        #se sale del cíclo el primer momento en que se rebasa el umbral
        else:
            #Este k-ésimo elemento es el primero con el cual se rebasa el umbral
            k_mejores.append(sig[k]) 
            break
    #se convierte la lista en un arreglo de numpy
    k_mejores = np.array(k_mejores)
    
    return k_mejores
            
